# RAG - Retrieval Augmented Generation
Trabalho final da disciplina de Topicos Contemporaneos

Alunos:
- Thiago Amorim: twa@cesar.school
- Arthur Wanderley: 

# Criação do vector store

Como o objetivo do trabalho é criar um sistema de RAG, escolhemos popular o vector store com informações fictícias, visto que o LLM não conseguiria utilizar seu conhecimento para responder as perguntas, e só conseguiria responder extraindo as informações do vector store.

Para isso, usamos o Claude 3.5 Sonnet para criar um conjunto de documentos fictícios sobre um robô biomimético para a exploração submarina.
Os documentos gerados estão na pasta `data/documents`. Eles serão separados para a população do vector store.

Analisando os documentos gerados, podemos perceber que eles possuem uma estrutura bem definida em `markdown`, e que cada seção não possui um tamanho muito grande, o que facilita a extração de informações, já que não excede a janela de contexto do LLM.

Por isso, optamos por criar cada documento separando a partir da seção de título do markdown (#).
Cada documento criado incluirá as subseções do documento original, e o título do documento será o título do documento original.



In [1]:
!pip install langchain
!pip install openai


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [38]:
import os

from langchain_core.documents import Document


def parse_document(content: str) -> list[Document]:
    """Cria documentos a partir do conteúdo de um arquivo.
    
    Exemplo:
    ```
    # Título do documento
    ## Seção 1
    Todo o texto vai para o documento 1
    ### Subseção 1.1
    Todo o texto vai para o documento 1
    ### Subseção 1.2
    Todo o texto vai para o documento 1
    ## Seção 2
    Todo o texto vai para o documento 2
    ### Subseção 2.1
    Todo o texto vai para o documento 2
    ```
    resultado:
    [
        Document(page_content='## Seção 1\nTodo o texto vai para o documento 1\n### Subseção 1.1\nTodo o texto vai para o documento 1\n### Subseção 1.2\nTodo o texto vai para o documento 1', metadata={'source': 'Título do documento'}),
        Document(page_content='## Seção 2\nTodo o texto vai para o documento 2\n### Subseção 2.1\nTodo o texto vai para o documento 2', metadata={'source': 'Título do documento'})
    ]
    """
    documents = []
    current_title = None
    current_content = ""

    for line in content.split('\n'):
        if line.startswith('## '):
            if current_title:
                documents.append(Document(page_content= "## " + current_title + '\n' + current_content, metadata={'source': current_title}))
                current_content = ""
            current_title = line[2:].strip()
        else:
            current_content += line + '\n'
    
    if current_title:
        documents.append(Document(page_content=current_content, metadata={'source': current_title}))
    
    return documents
    

#list all files in documents/
files = os.listdir('./documents/')

#load each file
documents = []
for file in files:
    with open(f'./documents/{file}', 'r') as f:
        documents.extend(parse_document(f.read()))

In [39]:
documents

[Document(metadata={'source': 'Projeto HidroNáutica X-700'}, page_content='## Projeto HidroNáutica X-700\n# Relatório Técnico Confidencial\n### Divisão de Robótica Submarina Avançada\n#### NeuroTech Oceanic Solutions\n**Versão do Documento: 1.4**\n**Data de Última Atualização: 15 de dezembro de 2037**\n\n'),
 Document(metadata={'source': '1. Introdução ao Projeto'}, page_content='## 1. Introdução ao Projeto\n\nO robô biomimético HidroNáutica X-700 representa um avanço revolucionário na exploração de ambientes submarinos extremos. Desenvolvido através de uma parceria interdisciplinar entre a NeuroTech Oceanic Solutions e o Instituto de Robótica Biomimética da Universidade Federal Marítima, o projeto visa criar um dispositivo de exploração que supere as limitações dos sistemas robóticos tradicionais.\n\n'),
 Document(metadata={'source': '2. Concepção Biomimética'}, page_content='## 2. Concepção Biomimética\n\nA inspiração principal para o X-700 veio do estudo detalhado de duas espécies m

In [32]:
!pip install faiss-cpu
!pip install python-dotenv
!pip install langchain-community
!pip install langchain-openai


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 7.9 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 16.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: openai
    Found existing installation: openai 1.53.0
    Uninstalling openai-1.53.0:
      Successfully uninstalled openai-1.53.0

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [40]:
from dotenv import load_dotenv

load_dotenv()

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

db = FAISS.from_documents(documents, OpenAIEmbeddings())

In [41]:
query = "Quando foi criado o projeto?"
docs = db.similarity_search(query)
docs


[Document(id='01f8dff1-76e6-4527-b4a0-e135ae2aa35b', metadata={'source': 'Projeto HidroNáutica X-700'}, page_content='## Projeto HidroNáutica X-700\n# Relatório Confidencial de Missão\n### Primeira Missão de Teste em Ambiente Extremo\n**Código da Missão:** NMEX-2038-001\n**Data da Missão:** 17 a 24 de março de 2038\n**Local:** Fossa das Marianas, Oceano Pacífico\n\n'),
 Document(id='25fbfc26-1ca5-4743-a06b-534109e156af', metadata={'source': 'Projeto HidroNáutica X-700'}, page_content='## Projeto HidroNáutica X-700\n# Relatório Técnico Confidencial\n### Divisão de Robótica Submarina Avançada\n#### NeuroTech Oceanic Solutions\n**Versão do Documento: 1.4**\n**Data de Última Atualização: 15 de dezembro de 2037**\n\n'),
 Document(id='4ae7e7d8-99d1-4277-9a23-6fc03de075a0', metadata={'source': '10. Equipe de Desenvolvimento'}, page_content='\n### 10.1 Liderança do Projeto\n\n#### Dra. Elena Rodrigues Silva\n**Cargo:** Diretora de Pesquisa e Coordenadora Geral do Projeto\n**Contribuições Princ

## Criando o RAG Agent

Agora que temos o vector store populado, podemos criar o RAG Agent.

In [60]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")


def retrieve_docs(query: str) -> list[Document]:
    documents = db.similarity_search(query)
    context = "---\n".join([doc.page_content for doc in documents])
    return context

system_prompt = """
Você é um assitente, e seu objetivo é responder as perguntas do usuário, somente com base no contexto abaixo.
Não utilize seu conhecimento para responder as perguntas, somente utilize o contexto fornecido.
Se não souber a resposta, diga que não sabe. Não tente inventar uma resposta.
A seguir está o contexto disponível para que você extraia a resposta: 
<contexto>
{context}
</contexto>

Agora, responda a seguinte pergunta:
<pergunta>
{query}
</pergunta>
"""

prompt_template = ChatPromptTemplate.from_template(system_prompt)

def ask_rag(query: str) -> str:
    context = retrieve_docs(query)
    prompt = prompt_template.invoke({"query": query, "context": context})
    return llm.invoke(prompt).content
    

from pprint import pprint

In [68]:
print(ask_rag("onde foi a primeira expedição do projeto?"))
print("\n----\n")
print(ask_rag("quem criou o robô?"))
print("\n----\n")
print(ask_rag("quem foi da equipe do projeto?"))
print("\n----\n")
print(ask_rag("o robô tem alguma limitação?"))


A primeira expedição do projeto foi na Fossa das Marianas, no Oceano Pacífico.

----

O robô biomimético HidroNáutica X-700 foi criado através de uma parceria interdisciplinar entre a NeuroTech Oceanic Solutions e o Instituto de Robótica Biomimética da Universidade Federal Marítima.

----

A equipe do projeto foi composta por:

1. **Dra. Elena Rodrigues Silva** - Diretora de Pesquisa e Coordenadora Geral do Projeto.
2. **Dr. Marcus Chen Wong** - Diretor Técnico de Engenharia.

----

Sim, o robô tem algumas limitações, como vulnerabilidade a campos eletromagnéticos intensos, tempo limitado de operação em zonas de temperatura extremamente baixa e dependência de calibração periódica dos sistemas neurais.

----

O contexto não fornece informações específicas sobre o tempo de operação do robô. Portanto, não sei a resposta.


Agora, vamos fazer algumas perguntas que não estão no contexto dos documentos

In [69]:
print("\n----\n")
print(ask_rag("por quanto tempo o robô pode operar?"))
print("\n----\n")
print(ask_rag("quem ganhou a copa do mundo de 1998?"))
print("\n----\n")
print(ask_rag("qual a altura do monte Everest?"))



----

O contexto não fornece informações específicas sobre o tempo exato de operação do robô. Apenas menciona que ele tem um "tempo limitado de operação em zonas de temperatura extremamente baixa". Portanto, não sabe responder com precisão sobre o tempo de operação do robô.

----

Não sei.

----

Não sei.
